In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import os
import gymnasium as gym
from gymnasium import spaces
from scipy.special import softmax
import matplotlib.pyplot as plt
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import SubprocVecEnv, VecNormalize
from sklearn.preprocessing import MinMaxScaler
import torch as th
import sys
import os

2025-03-19 11:23:37.902169: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from scipy.stats import t as student_t
import talib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import sys
import os

# Instead of importing, copy the SimpleOHLCGenerator class 
# that you shared earlier into your current file
# Or use this simpler import:
sys.path.append("/Users/newuser/Projects/robust_algo_trader/drl/")
from ohlc_generator import SimpleOHLCGenerator  # Remove 'drl.' prefix

In [ ]:
if __name__ == "__main__":
    df = pd.read_csv("/Users/newuser/Projects/robust_algo_trader/data/gen_alpaca_data/AAPL_D1_raw_data.csv")
    generator = SimpleOHLCGenerator(df)
    synthetic_data = generator.generate_bootstrap_data(num_samples=10, segment_length=5)[1]
    fig = generator.plot_comparison(synthetic_data, num_bars=130)
    plt.show()

In [ ]:
synthetic_data